In [1]:
colab = 1
if colab :
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Github/Opti-Project/src/vision_experiments
  !pip install einops

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/Opti-Project/src/vision_experiments


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import random
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import functools
from torch.optim.lr_scheduler import LambdaLR
from functools import partial
from pathlib import Path
from PIL import Image
from train_utils import *
from vit_small import *

In [4]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#32 42 73
set_seed(42)

cuda:0


In [9]:
def train_eval_model(model, model_name, forward, optimizer, clip_val=0) :
    nb_epochs = 30
    input_size = train_loader.dataset[0][0].nelement()
    num_classes = 10
    model.to(device)
    print(model_name)
    k_value = 1e-4
    lr_lambda = functools.partial(exponential_lr_decay, k=k_value)
    scheduler = LambdaLR(optimizer, lr_lambda)
    #scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
    output_path = Path("visionModels_runs/forward/") if forward else Path("visionModels_runs/backward/")
    if forward :
      _, _, _, _, run_path = train_validate_forward(
        device,
        nb_epochs,
        model,
        model_name,
        input_size,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        output_path,
        gradient_clip_val=clip_val,
        flatten=False)
    else :
        _, _, _, _, run_path = train_validate_backward(
        device,
        nb_epochs,
        model,
        model_name,
        input_size,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        output_path,
        gradient_clip_val=clip_val,
        flatten=False)
    test_results = evaluate(device, model, input_size, test_loader, run_path=run_path, flatten=False)
    print("test results")
    print(test_results)

In [6]:
train_loader, val_loader, test_loader = get_data("CIFAR10", img_size=32)

Files already downloaded and verified
Files already downloaded and verified
Train dataset: 45000 batch size 64 of dim torch.Size([3, 32, 32])
Validation dataset: 5000 batch size 64 of dim torch.Size([3, 32, 32])
Test dataset: 10000 batch size 64 of dim torch.Size([3, 32, 32])


# Basic convnet

In [10]:
class ConvNet(torch.nn.Module):
    def __init__(self, input_size: int = 1, output_size: int = 10):
        """Standard Convolutional Network layers.

        Args:
            input_size (int): input size of the model.
            output_size (int): The number of output classes.

        """
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(input_size, 64, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(4096, 1024)
        self.fc2 = torch.nn.Linear(1024, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Backward

In [11]:
model_name = "convnet"
model = ConvNet(input_size = 3, output_size= 10)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, False, optimizer, clip_val=0)

convnet


	Validating: 100%|██████████| 79/79 [00:01<00:00, 46.00it/s, iters_per_sec=46, vacc=0.75, vf1=0.583, vloss=0.945]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.5021 Val Loss: 1.1792 Train Acc: 0.4606 Val Acc: 0.5710 Train F1: 0.4277 Val F1: 0.5394
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 45.83it/s, iters_per_sec=45.8, vacc=0.75, vf1=0.63, vloss=0.75]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 1.0628 Val Loss: 0.9650 Train Acc: 0.6252 Val Acc: 0.6549 Train F1: 0.5997 Val F1: 0.6347
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 0.0186 Val Loss: 1.8629 Train Acc: 0.9937 Val Acc: 0.7387 Train F1: 0.9933 Val F1: 0.7219


	Validating: 100%|██████████| 79/79 [00:01<00:00, 46.23it/s, iters_per_sec=46.2, vacc=0.625, vf1=0.548, vloss=2.32]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 0.0178 Val Loss: 1.9413 Train Acc: 0.9941 Val Acc: 0.7364 Train F1: 0.9934 Val F1: 0.7168


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 45.70it/s, eval_acc=0.75, eval_f1=0.606, iters_per_sec=45.7]


test results
{'Acc': 0.7378582802547771, 'F1': 0.7177551995248344}


## Forward

In [12]:
model_name = "convnet"
model = ConvNet(input_size = 3, output_size= 10)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=0)

convnet


	Validating: 100%|██████████| 79/79 [00:01<00:00, 43.62it/s, iters_per_sec=43.6, vacc=0, vf1=0, vloss=2.3]


Epoch 1 lr: 0.0002 Averages : Train Loss: 2.3015 Val Loss: 2.3006 Train Acc: 0.1010 Val Acc: 0.0977 Train F1: 0.0452 Val F1: 0.0375
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 44.56it/s, iters_per_sec=44.5, vacc=0.25, vf1=0.0667, vloss=2.28]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 2.2981 Val Loss: 2.2962 Train Acc: 0.1184 Val Acc: 0.1224 Train F1: 0.0596 Val F1: 0.0575
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 1.9628 Val Loss: 1.9774 Train Acc: 0.2978 Val Acc: 0.2834 Train F1: 0.2702 Val F1: 0.2601


	Validating: 100%|██████████| 79/79 [00:01<00:00, 45.91it/s, iters_per_sec=45.9, vacc=0.625, vf1=0.521, vloss=1.67]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 1.9590 Val Loss: 1.9702 Train Acc: 0.3003 Val Acc: 0.2955 Train F1: 0.2725 Val F1: 0.2660
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 45.26it/s, eval_acc=0.438, eval_f1=0.274, iters_per_sec=45.3]

test results
{'Acc': 0.30642914012738853, 'F1': 0.27431730647792685}


In [13]:
model_name = "convnet"
model = ConvNet(input_size = 3, output_size= 10)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=5)

convnet


	Validating: 100%|██████████| 79/79 [00:01<00:00, 45.95it/s, iters_per_sec=45.9, vacc=0, vf1=0, vloss=2.29]


Epoch 1 lr: 0.0002 Averages : Train Loss: 2.3015 Val Loss: 2.2997 Train Acc: 0.1139 Val Acc: 0.1060 Train F1: 0.0449 Val F1: 0.0335
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 45.11it/s, iters_per_sec=45.1, vacc=0.375, vf1=0.171, vloss=2.29]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 2.2990 Val Loss: 2.2975 Train Acc: 0.1136 Val Acc: 0.1345 Train F1: 0.0437 Val F1: 0.0561
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 2.0342 Val Loss: 2.0564 Train Acc: 0.2750 Val Acc: 0.2633 Train F1: 0.2474 Val F1: 0.2366


	Validating: 100%|██████████| 79/79 [00:01<00:00, 45.89it/s, iters_per_sec=45.9, vacc=0.375, vf1=0.3, vloss=2.1]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 2.0339 Val Loss: 2.0484 Train Acc: 0.2665 Val Acc: 0.2573 Train F1: 0.2390 Val F1: 0.2357
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 45.35it/s, eval_acc=0.25, eval_f1=0.144, iters_per_sec=45.3]

test results
{'Acc': 0.2643312101910828, 'F1': 0.2358627651725982}


# Resnet

## Backward

In [14]:
model_name = "resnet18"
model = models.resnet18(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, False, optimizer, clip_val=0)

resnet18


	Validating: 100%|██████████| 79/79 [00:01<00:00, 40.53it/s, iters_per_sec=40.5, vacc=0.5, vf1=0.375, vloss=0.997]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.5854 Val Loss: 1.2683 Train Acc: 0.4692 Val Acc: 0.5352 Train F1: 0.4461 Val F1: 0.5091
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 40.51it/s, iters_per_sec=40.5, vacc=0.375, vf1=0.241, vloss=1.96]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 1.0780 Val Loss: 1.0725 Train Acc: 0.6149 Val Acc: 0.6084 Train F1: 0.5960 Val F1: 0.5873
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 0.0645 Val Loss: 1.7109 Train Acc: 0.9774 Val Acc: 0.6940 Train F1: 0.9739 Val F1: 0.6734


	Validating: 100%|██████████| 79/79 [00:01<00:00, 40.01it/s, iters_per_sec=40, vacc=0.875, vf1=0.8, vloss=1.46]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 0.0609 Val Loss: 1.6399 Train Acc: 0.9786 Val Acc: 0.7027 Train F1: 0.9758 Val F1: 0.6849


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 40.45it/s, eval_acc=0.688, eval_f1=0.467, iters_per_sec=40.4]

test results
{'Acc': 0.7005374203821656, 'F1': 0.682226337510687}


## Forward

In [15]:
model_name = "resnet18"
model = models.resnet18(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=0)

resnet18


	Validating: 100%|██████████| 79/79 [00:01<00:00, 39.77it/s, iters_per_sec=39.8, vacc=0, vf1=0, vloss=6.98]


Epoch 1 lr: 0.0002 Averages : Train Loss: 7.0509 Val Loss: 6.9264 Train Acc: 0.0004 Val Acc: 0.0016 Train F1: 0.0001 Val F1: 0.0005
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 39.59it/s, iters_per_sec=39.6, vacc=0, vf1=0, vloss=6.69]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 6.8158 Val Loss: 6.7318 Train Acc: 0.0014 Val Acc: 0.0026 Train F1: 0.0004 Val F1: 0.0008
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 3.5136 Val Loss: 3.4876 Train Acc: 0.1474 Val Acc: 0.1416 Train F1: 0.1333 Val F1: 0.1298
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 40.54it/s, iters_per_sec=40.5, vacc=0.25, vf1=0.175, vloss=3.17]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 3.4795 Val Loss: 3.4631 Train Acc: 0.1444 Val Acc: 0.1442 Train F1: 0.1290 Val F1: 0.1294
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 41.60it/s, eval_acc=0.125, eval_f1=0.0815, iters_per_sec=41.6]

test results
{'Acc': 0.14380971337579618, 'F1': 0.1301014180549576}


In [16]:
model_name = "resnet18"
model = models.resnet18(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=5)

resnet18


	Validating: 100%|██████████| 79/79 [00:02<00:00, 38.66it/s, iters_per_sec=38.6, vacc=0, vf1=0, vloss=6.91]


Epoch 1 lr: 0.0002 Averages : Train Loss: 6.9524 Val Loss: 6.8927 Train Acc: 0.0006 Val Acc: 0.0008 Train F1: 0.0002 Val F1: 0.0003
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 39.92it/s, iters_per_sec=39.9, vacc=0, vf1=0, vloss=6.79]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 6.8288 Val Loss: 6.7676 Train Acc: 0.0015 Val Acc: 0.0016 Train F1: 0.0005 Val F1: 0.0005
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 3.9927 Val Loss: 3.9663 Train Acc: 0.1387 Val Acc: 0.1327 Train F1: 0.1164 Val F1: 0.1129
Best model saved


	Validating: 100%|██████████| 79/79 [00:01<00:00, 40.44it/s, iters_per_sec=40.4, vacc=0.125, vf1=0.111, vloss=4.19]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 3.9357 Val Loss: 3.9240 Train Acc: 0.1418 Val Acc: 0.1390 Train F1: 0.1196 Val F1: 0.1180
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:03<00:00, 41.27it/s, eval_acc=0.0625, eval_f1=0.05, iters_per_sec=41.3]

test results
{'Acc': 0.1439092356687898, 'F1': 0.12103083545413078}


# small Vit

## backward

In [17]:
model_name = "small_vit"
model = ViT_small(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 256,
    dropout = 0.1,
    emb_dropout = 0.1)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, False, optimizer, clip_val=0)

small_vit


	Validating: 100%|██████████| 79/79 [00:02<00:00, 32.28it/s, iters_per_sec=32.3, vacc=0.25, vf1=0.146, vloss=1.91]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.5279 Val Loss: 1.2865 Train Acc: 0.4395 Val Acc: 0.5366 Train F1: 0.4041 Val F1: 0.5089
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 32.16it/s, iters_per_sec=32.1, vacc=0.5, vf1=0.396, vloss=1.03]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 1.1721 Val Loss: 1.2181 Train Acc: 0.5802 Val Acc: 0.5667 Train F1: 0.5551 Val F1: 0.5400
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 0.0858 Val Loss: 1.8696 Train Acc: 0.9689 Val Acc: 0.6388 Train F1: 0.9659 Val F1: 0.6180


	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.82it/s, iters_per_sec=31.8, vacc=0.75, vf1=0.583, vloss=0.958]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 0.0719 Val Loss: 1.9768 Train Acc: 0.9751 Val Acc: 0.6410 Train F1: 0.9723 Val F1: 0.6191


	Evaluating: 100%|██████████| 157/157 [00:04<00:00, 32.25it/s, eval_acc=0.812, eval_f1=0.853, iters_per_sec=32.2]

test results
{'Acc': 0.6448049363057324, 'F1': 0.6209136092064651}


## forward

In [18]:
model_name = "small_vit"
model = ViT_small(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 256,
    dropout = 0.1,
    emb_dropout = 0.1)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=0)

small_vit


	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.01it/s, iters_per_sec=31, vacc=0.125, vf1=0.0476, vloss=2.29]


Epoch 1 lr: 0.0002 Averages : Train Loss: 2.3490 Val Loss: 2.3138 Train Acc: 0.1034 Val Acc: 0.1153 Train F1: 0.0656 Val F1: 0.0948
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.14it/s, iters_per_sec=31.1, vacc=0, vf1=0, vloss=2.38]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 2.3043 Val Loss: 2.2992 Train Acc: 0.1208 Val Acc: 0.1270 Train F1: 0.1061 Val F1: 0.1143
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 2.0904 Val Loss: 2.0835 Train Acc: 0.2123 Val Acc: 0.2227 Train F1: 0.1911 Val F1: 0.2008


	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.07it/s, iters_per_sec=31.1, vacc=0, vf1=0, vloss=2.06]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 2.0939 Val Loss: 2.0811 Train Acc: 0.2135 Val Acc: 0.2174 Train F1: 0.1918 Val F1: 0.1993


	Evaluating: 100%|██████████| 157/157 [00:04<00:00, 32.77it/s, eval_acc=0.375, eval_f1=0.205, iters_per_sec=32.8]


test results
{'Acc': 0.24343152866242038, 'F1': 0.21101735732320007}


In [19]:
model_name = "small_vit"
model = ViT_small(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 256,
    dropout = 0.1,
    emb_dropout = 0.1)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
train_eval_model(model, model_name, True, optimizer, clip_val=5)

small_vit


	Validating: 100%|██████████| 79/79 [00:02<00:00, 30.88it/s, iters_per_sec=30.9, vacc=0, vf1=0, vloss=2.4]


Epoch 1 lr: 0.0002 Averages : Train Loss: 2.3250 Val Loss: 2.2982 Train Acc: 0.1337 Val Acc: 0.1377 Train F1: 0.0890 Val F1: 0.1049
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 30.96it/s, iters_per_sec=30.9, vacc=0.125, vf1=0.0833, vloss=2.36]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 2.2881 Val Loss: 2.2734 Train Acc: 0.1367 Val Acc: 0.1519 Train F1: 0.1125 Val F1: 0.1239
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 2.0948 Val Loss: 2.0993 Train Acc: 0.2078 Val Acc: 0.2138 Train F1: 0.1849 Val F1: 0.1909


	Validating: 100%|██████████| 79/79 [00:02<00:00, 30.96it/s, iters_per_sec=30.9, vacc=0.125, vf1=0.0556, vloss=1.96]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 2.0959 Val Loss: 2.0922 Train Acc: 0.2110 Val Acc: 0.2027 Train F1: 0.1865 Val F1: 0.1789
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:04<00:00, 32.34it/s, eval_acc=0.125, eval_f1=0.0833, iters_per_sec=32.3]

test results
{'Acc': 0.24233678343949044, 'F1': 0.19853848452082093}
